In [86]:
import os, re, datetime
import pandas as pd
from collections import OrderedDict 
from docxtpl import DocxTemplate
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import RGBColor
from docx.api import Document

In [87]:
df = pd.read_excel('/Users/andy/Desktop/化疗药/泛癌种50基因/化疗综合数据库.xlsx', sheet_name='500基因化疗部分')

In [88]:
df = df.dropna(axis = 0, subset = ['类别'] )
#填充nan值的函数
def make_column(string):
    a = ''
    list1 = []
    for i in df.index:
        if type(df[string][i]) != type(df[string][2]):
            a = df[string][i]
            list1.append(a)
        else:
            list1.append(a)
    return list1

for i in ['化疗药物', '基因', '位点', '证据等级']:
    df[i] = (make_column(i))

In [89]:
dic = {}
dic1 = {}
for i in df.index:
    dic.setdefault(df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i], {})
    dic1.setdefault(df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i], {})

    dic[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]][df['基因型'][i]] = df['解释'][i]
    dic1[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]][df['基因型'][i]] = df['类别'][i]
    dic1[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]]['证据等级'] = df['证据等级'][i]

In [90]:
list1 = ['rs1801133','rs1801019','rs67376798','rs4148323' ,'rs1056892','rs3212986','rs4353229','rs1800566' ]
list2 = ['rs25487', 'rs11615','rs11545078']

list3 = []

In [91]:
def modify_result(df_name, list1, list2, list3):
    #list1:基因型与数据库相反的位点列表，list2:基因型与数据库反向互补的位点列表 
    #list3:基因型与数据库互补的位点列表
    reverse_complement = lambda gt: gt.replace('A','t').replace('T','a').replace('G','c').replace('C','g').upper()[::-1]
    complement = lambda gt: gt.replace('A','t').replace('T','a').replace('G','c').replace('C','g').upper()
    for i in range(df_name.shape[0]):
        for line1 in list1:
            df_name.at[i, line1] = df_client.at[i, line1][::-1]#将检测结果中与数据库相反的基因型校正
        for line2 in list2:
            df_name.at[i, line2] = reverse_complement(df_name.at[i, line2])
        for line3 in list3:
            df_name.at[i, line3] = complement(df_name.at[i, line3])

In [92]:
df_client = pd.read_excel('/Users/andy/Desktop/化疗药/泛癌种500基因化疗/实体瘤500基因.xlsx')
modify_result(df_client, list1, list2, list3)
list_rs = re.findall(r'rs\d+', str(df_client.columns))
dic_client = {}
for i in range(df_client.shape[0]):
    for j in list_rs:
        dic_client.setdefault (df_client['姓名'][i], {})
        dic_client[df_client['姓名'][i]][j] = df_client[j][i]

In [93]:
"""
level_dic = {'1A':6 , '1B':5, '2A':4, '2B':3, '3':2, '4':1 }
#输入客户名字，产生临床提示结果
def clinical_prompt(client_name ):
    d_result = {}#临床提示映射
    list_pinggu = []
    d_test = {}
    dic_aa = {}
    for i in dic1.keys():
        list_pinggu.append(dic1[i][dic_client[client_name][i.split('\t')[2]]] + '--' + str(dic1[i]['证据等级']) )
        dic_pinggu_result = OrderedDict(zip(dic1.keys(), list_pinggu))
    for i in dic_pinggu_result.keys():
        d_test.setdefault(i.split('\t')[0],[])
        d_test[i.split('\t')[0]].append(dic_pinggu_result[i])
    
    
    for j, k in d_test.items():
        target = []
        for line in k:
            for z in line.split('，'):
                if '--' not in z:
                    target.append(z + '--' + line.split('--')[1])
                else:
                    target.append(z)
    
        dic_aa[j] = target
    
    
    for i, a in dic_aa.items():
        list_effect = []
        
        du = re.findall(r'毒副\w+--\w+', str(a))
        dai = re.findall(r'代谢\w+--\w+', str(a))
        liao = re.findall(r'疗效\w+--\w+', str(a))
        
        top_num_liao = re.findall(r'较好--\w+', str(liao))
        mid_num_liao = re.findall(r'适中--\w+', str(liao))
        bot_num_liao = re.findall(r'较差--\w+', str(liao))
        
        top_num_du = re.findall(r'较高--\w+', str(du))
        mid_num_du = re.findall(r'适中--\w+', str(du))
        bot_num_du = re.findall(r'较低--\w+', str(du))
        
        top_num_dai = re.findall(r'较快--\w+', str(dai))
        mid_num_dai = re.findall(r'适中--\w+', str(dai))
        bot_num_dai = re.findall(r'较慢--\w+', str(dai))
        
        
        
        #疗效结果判定
        if len(liao) == 1: #位点数目为1的时候
            if '适中' in liao[0] and level_dic[liao[0].split('--')[1]] >= level_dic['2A'] or  '较好' in liao[0]:
                list_effect.append('疗效较好')
            if '适中' in liao[0] and level_dic[liao[0].split('--')[1]] < level_dic['2A'] or  '较差' in liao[0]:
                list_effect.append('疗效适中')
        
        if len(liao) == 2:
            if len(top_num_liao) == 2 or len(top_num_liao ) == 1 and len(mid_num_liao ) == 1:
                list_effect.append('疗效较好')
            if len(top_num_liao) == 1 and len(bot_num_liao) == 1 or len(mid_num_liao) == 2 or len(mid_num_liao) == 1 and len(bot_num_liao) == 1 \
            or len(bot_num_liao) == 2:
                list_effect.append('疗效适中')
        
        if len(liao) == 3:
            if len(top_num_liao) == 3 or len(top_num_liao) == 2 and len(mid_num_liao) == 1 or len(top_num_liao) == 2 \
            and len(bot_num_liao) == 1 or len(top_num_liao) == 1 and len(mid_num_liao) ==2 or len(top_num_liao) == 1 \
            and len(mid_num_liao) == 1 and len(bot_num_liao) == 1:
                list_effect.append('疗效较好')
            if  len(top_num_liao) == 1 and len(bot_num_liao) == 2 or len(mid_num_liao) == 3 or len(mid_num_liao) == 2 \
            and len(bot_num_liao) == 1 or len(mid_num_liao) == 1 and len(bot_num_liao) == 2 or len(bot_num_liao) == 3:
                list_effect.append('疗效适中')
        
        
        
        
        #毒副结果判定
        if i == '氟尿嘧啶+亚叶酸':
            if set(a) == set(['毒副较低 --1A', '毒副较低--2B', '毒副较高--3']) or set(a) == set(['毒副较低 --1A', '毒副较低--2B', '毒副较低--3']):
                    list_effect.append('毒副较低')
            
                
            if set(a) == set(['毒副较低 --1A', '毒副较高--2B', '毒副较高--3']) or set(a) == set(['毒副较低 --1A', '毒副较高--2B', '毒副较低--3']):
                    list_effect.append('毒副适中')
            
                    
            
            
            
                
                
        else:
            if len(du) == 1:
                if '较高' in du[0]:
                    list_effect.append('毒副较高')
                if '适中' in du[0]:
                    list_effect.append('毒副适中')
                if '较低' in du[0]:
                    list_effect.append('毒副较低')
        
            if len(du) == 2:    
                if len(top_num_du) == 2 or len(top_num_du) == 1 and len(mid_num_du) == 1 or len(top_num_du) == 1 and \
                level_dic[top_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 1:
                    list_effect.append('毒副较高')
                if len(top_num_du) == 1 and level_dic[top_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) == 1 or len(mid_num_du) \
                == 2 or len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] >= level_dic['2A'] and len(bot_num_du) \
                == 1:
                    list_effect.append('毒副适中')
                if len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] <= level_dic['2B'] and len(bot_num_du) == 1 \
                or len(bot_num_du) == 2:
                    list_effect.append('毒副较低')
            
            if len(du) == 3:
                if len(top_num_du) == 3 or len(top_num_du) == 2 and len(mid_num_du) == 1 or len(top_num_du) == 1 and len(mid_num_du) \
                == 2 or len(top_num_du) == len(mid_num_du) == len(bot_num_du) == 1 or len(top_num_du) == 1 and \
                level_dic[top_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 2:
                    list_effect.append('毒副较高')
                if len(top_num_du) == 1 and level_dic[top_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) \
                == 2 or len(mid_num_du) == 3 or len(mid_num_du) == 2 and len(bot_num_du) == 1 or len(mid_num_du) == 1 \
                and level_dic[mid_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 2:
                    list_effect.append('毒副适中')
                if len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) == 2 \
                or len(bot_num_du) == 3:
                    list_effect.append('毒副较低')
            
            if len(du) == 4:
                if set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较高--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副适中--2A','毒副较高--3'}:
                    list_effect.append('毒副适中')
                if set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较低--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副适中--2A','毒副较低--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较低--2A','毒副较高--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较低--2A','毒副适中--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较低--2A','毒副较低--3'}:
                    list_effect.append('毒副较低')
                
            if len(du) == 5:
                if set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较高--3','毒副较高--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较高--3','毒副适中--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较高--3','毒副较低--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副适中--2A','毒副较高--3','毒副较高--3'}:
                    list_effect.append('毒副适中')
                else:
                    list_effect.append('毒副较低')

        
        
        #代谢结果判定
        if len(dai) == 1:
            if '较快' in dai[0]:
                list_effect.append('代谢较快')
            if '适中' in dai[0]:
                list_effect.append('代谢适中')
            if '较慢' in dai[0]:
                list_effect.append('代谢较慢')
        
        if len(dai) == 2:
            if len(top_num_dai) == 2 or len(top_num_dai) == 1 and len(mid_num_dai) == 1:
                list_effect.append('代谢较快')
            if len(top_num_dai) == 1 and len(bot_num_dai) == 1 and level_dic[bot_num_dai[0].split('--')[1]] \
            < level_dic['1A'] or len(mid_num_dai) == 2:
                list_effect.append('代谢适中')
            if len(mid_num_dai) == 1 and len(bot_num_dai) == 1 or len(bot_num_dai) == 2 or len(top_num_dai) \
            == 1 and len(bot_num_dai) == 1 and level_dic[bot_num_dai[0].split('--')[1]] == level_dic['1A']:
                list_effect.append('代谢较慢')
    
    
        d_result[i] = '，'.join(list_effect)
        
    return d_result
"""

"\nlevel_dic = {'1A':6 , '1B':5, '2A':4, '2B':3, '3':2, '4':1 }\n#输入客户名字，产生临床提示结果\ndef clinical_prompt(client_name ):\n    d_result = {}#临床提示映射\n    list_pinggu = []\n    d_test = {}\n    dic_aa = {}\n    for i in dic1.keys():\n        list_pinggu.append(dic1[i][dic_client[client_name][i.split('\t')[2]]] + '--' + str(dic1[i]['证据等级']) )\n        dic_pinggu_result = OrderedDict(zip(dic1.keys(), list_pinggu))\n    for i in dic_pinggu_result.keys():\n        d_test.setdefault(i.split('\t')[0],[])\n        d_test[i.split('\t')[0]].append(dic_pinggu_result[i])\n    \n    \n    for j, k in d_test.items():\n        target = []\n        for line in k:\n            for z in line.split('，'):\n                if '--' not in z:\n                    target.append(z + '--' + line.split('--')[1])\n                else:\n                    target.append(z)\n    \n        dic_aa[j] = target\n    \n    \n    for i, a in dic_aa.items():\n        list_effect = []\n        \n        du = re.findall(r'毒副\\w

In [94]:
level_dic = {'1A':6 , '1B':5, '2A':4, '2B':3, '3':2, '4':1 }
#输入客户名字，产生临床提示结果
def clinical_prompt(client_name ):
    d_result = {}#临床提示映射
    list_pinggu = []
    d_test = {}
    dic_aa = {}
    for i in dic1.keys():
        list_pinggu.append(dic1[i][dic_client[client_name][i.split('\t')[2]]] + '--' + str(dic1[i]['证据等级']) )
        dic_pinggu_result = OrderedDict(zip(dic1.keys(), list_pinggu))
    for i in dic_pinggu_result.keys():
        d_test.setdefault(i.split('\t')[0],[])
        d_test[i.split('\t')[0]].append(dic_pinggu_result[i])
    
    
    for j, k in d_test.items():
        target = []
        for line in k:
            for z in line.split('，'):
                if '--' not in z:
                    target.append(z + '--' + line.split('--')[1])
                else:
                    target.append(z)
    
        dic_aa[j] = target
    
    
    for i, a in dic_aa.items():
        list_effect = []
        
        du = re.findall(r'毒副\w+--\w+', str(a))
        dai = re.findall(r'代谢\w+--\w+', str(a))
        liao = re.findall(r'疗效\w+--\w+', str(a))
        
        top_num_liao = re.findall(r'较好--\w+', str(liao))
        mid_num_liao = re.findall(r'适中--\w+', str(liao))
        bot_num_liao = re.findall(r'较差--\w+', str(liao))
        
        top_num_du = re.findall(r'较高--\w+', str(du))
        mid_num_du = re.findall(r'适中--\w+', str(du))
        bot_num_du = re.findall(r'较低--\w+', str(du))
        
        top_num_dai = re.findall(r'较快--\w+', str(dai))
        mid_num_dai = re.findall(r'适中--\w+', str(dai))
        bot_num_dai = re.findall(r'较慢--\w+', str(dai))
        
        
        
        #疗效结果判定
        if len(liao) == 1: #位点数目为1的时候
            if '适中' in liao[0] and level_dic[liao[0].split('--')[1]] >= level_dic['2A'] or  '较好' in liao[0]:
                list_effect.append('疗效较好')
            if '适中' in liao[0] and level_dic[liao[0].split('--')[1]] < level_dic['2A'] or  '较差' in liao[0]:
                list_effect.append('疗效适中')
        
        if len(liao) == 2:
            if len(top_num_liao) == 2 or len(top_num_liao ) == 1 and len(mid_num_liao ) == 1:
                list_effect.append('疗效较好')
            if len(top_num_liao) == 1 and len(bot_num_liao) == 1 or len(mid_num_liao) == 2 or len(mid_num_liao) == 1 and len(bot_num_liao) == 1 \
            or len(bot_num_liao) == 2:
                list_effect.append('疗效适中')
        
        if len(liao) == 3:
            if len(top_num_liao) == 3 or len(top_num_liao) == 2 and len(mid_num_liao) == 1 or len(top_num_liao) == 2 \
            and len(bot_num_liao) == 1 or len(top_num_liao) == 1 and len(mid_num_liao) ==2 or len(top_num_liao) == 1 \
            and len(mid_num_liao) == 1 and len(bot_num_liao) == 1:
                list_effect.append('疗效较好')
            if  len(top_num_liao) == 1 and len(bot_num_liao) == 2 or len(mid_num_liao) == 3 or len(mid_num_liao) == 2 \
            and len(bot_num_liao) == 1 or len(mid_num_liao) == 1 and len(bot_num_liao) == 2 or len(bot_num_liao) == 3:
                list_effect.append('疗效适中')
        
        
        
        
        #毒副结果判定
        if i == '氟尿嘧啶+亚叶酸':
            if set(du) == set(['毒副较低 --1A', '毒副较低--2B', '毒副较高--3']) or set(du) == set(['毒副较低 --1A', '毒副较低--2B', '毒副较低--3']):
                    list_effect.append('毒副较低')
            
                
            if set(du) == set(['毒副较低 --1A', '毒副较高--2B', '毒副较高--3']) or set(du) == set(['毒副较低 --1A', '毒副较高--2B', '毒副较低--3']):
                    list_effect.append('毒副适中')
        

        if i == '氟尿嘧啶':
            if set(du) == set(['毒副较低--1A', '毒副较低--1A', '毒副较高--3']) or set(du) == set(['毒副较低--1A', '毒副较低--1A', '毒副适中--3']) or \
            set(du) == set(['毒副较低--1A', '毒副较低--1A', '毒副较低--3']):
                    list_effect.append('毒副较低')
        
       







            
                    
            
            
            
                
                
        else:
            if len(du) == 1:
                if '较高' in du[0]:
                    list_effect.append('毒副较高')
                if '适中' in du[0]:
                    list_effect.append('毒副适中')
                if '较低' in du[0]:
                    list_effect.append('毒副较低')
        
            if len(du) == 2:    
                if len(top_num_du) == 2 or len(top_num_du) == 1 and len(mid_num_du) == 1 or len(top_num_du) == 1 and \
                level_dic[top_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 1:
                    list_effect.append('毒副较高')
                if len(top_num_du) == 1 and level_dic[top_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) == 1 or len(mid_num_du) \
                == 2 or len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] >= level_dic['2A'] and len(bot_num_du) \
                == 1:
                    list_effect.append('毒副适中')
                if len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] <= level_dic['2B'] and len(bot_num_du) == 1 \
                or len(bot_num_du) == 2:
                    list_effect.append('毒副较低')
            
            if len(du) == 3:
                if len(top_num_du) == 3 or len(top_num_du) == 2 and len(mid_num_du) == 1 or len(top_num_du) == 2 and len(bot_num_du) == 1 or len(top_num_du) == 1 and len(mid_num_du) \
                == 2 or len(top_num_du) == len(mid_num_du) == len(bot_num_du) == 1 or len(top_num_du) == 1 and \
                level_dic[top_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 2:
                    list_effect.append('毒副较高')
                if len(top_num_du) == 1 and level_dic[top_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) \
                == 2 or len(mid_num_du) == 3 or len(mid_num_du) == 2 and len(bot_num_du) == 1 or len(mid_num_du) == 1 \
                and level_dic[mid_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 2:
                    list_effect.append('毒副适中')
                if len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) == 2 \
                or len(bot_num_du) == 3:
                    list_effect.append('毒副较低')
            
            if len(du) == 4:
                if set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较高--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副适中--2A','毒副较高--3'}:
                    list_effect.append('毒副适中')
                if set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较低--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副适中--2A','毒副较低--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较低--2A','毒副较高--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较低--2A','毒副适中--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较低--2A','毒副较低--3'}:
                    list_effect.append('毒副较低')
                
            if len(du) == 5:
                if set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较高--3','毒副较高--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较高--3','毒副适中--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副较高--2A','毒副较高--3','毒副较低--3'} or \
                set(du) == {'毒副较低--1A','毒副较低--1A','毒副适中--2A','毒副较高--3','毒副较高--3'}:
                    list_effect.append('毒副适中')
                else:
                    list_effect.append('毒副较低')

        
        
        #代谢结果判定
        if len(dai) == 1:
            if '较快' in dai[0]:
                list_effect.append('代谢较快')
            if '适中' in dai[0]:
                list_effect.append('代谢适中')
            if '较慢' in dai[0]:
                list_effect.append('代谢较慢')
        
        if len(dai) == 2:
            if len(top_num_dai) == 2 or len(top_num_dai) == 1 and len(mid_num_dai) == 1:
                list_effect.append('代谢较快')
            if len(top_num_dai) == 1 and len(bot_num_dai) == 1 and level_dic[bot_num_dai[0].split('--')[1]] \
            < level_dic['1A'] or len(mid_num_dai) == 2:
                list_effect.append('代谢适中')
            if len(mid_num_dai) == 1 and len(bot_num_dai) == 1 or len(bot_num_dai) == 2 or len(top_num_dai) \
            == 1 and len(bot_num_dai) == 1 and level_dic[bot_num_dai[0].split('--')[1]] == level_dic['1A']:
                list_effect.append('代谢较慢')
    
    
        d_result[i] = '，'.join(list_effect)
        
    return d_result
                


In [95]:
def gene_type_level(name):
    gene_type_dic = {}
    for i in dic1.keys():
        gene_type_dic.setdefault(i, {})
        gene_type_dic[i]['基因型'] = dic_client[name][i.split('\t')[-1]]
        gene_type_dic[i]['证据等级'] = dic1[i]['证据等级']
        text = dic1[i][dic_client[name][i.split('\t')[2]]]
        dai = re.findall(r'代谢', text)
        du = re.findall(r'毒副', text)
        liao = list(set(re.findall(r'疗效', text)))
        x = []
        for k in [liao, du, dai]:
            if k:
                x += k
        #print('，'.join(x).replace('副', '性'))
        
        

        gene_type_dic[i]['类别'] = '，'.join(x).replace('副', '性')
            
        
    return gene_type_dic

In [96]:
dic_info_client = {}
for i in range(df_client.shape[0]):
    dic_info_client.setdefault(df_client['姓名'][i], {})
    dic_info_client[df_client['姓名'][i]]['采样日期'] = str(df_client['采样时间'][i])[:4] + '-' + str(df_client['采样时间'][i])[4:6] + '-' + str(df_client['采样时间'][i])[6:8]
    dic_info_client[df_client['姓名'][i]]['收样日期'] = str(df_client['收样时间'][i])[:4] + '-' + str(df_client['收样时间'][i])[4:6] + '-' + str(df_client['收样时间'][i])[6:8]
    dic_info_client[df_client['姓名'][i]]['姓名'] = df_client['姓名'][i]
    dic_info_client[df_client['姓名'][i]]['性别'] = df_client['性别'][i]
    dic_info_client[df_client['姓名'][i]]['年龄'] = df_client['年龄'][i]
    dic_info_client[df_client['姓名'][i]]['样本编号'] = df_client['实验室编号'][i]
    dic_info_client[df_client['姓名'][i]]['临床提示'] = clinical_prompt(df_client['姓名'][i])
    dic_info_client[df_client['姓名'][i]]['基因型'] = gene_type_level(df_client['姓名'][i])
    dt = datetime.date.today()
    reportdate = '{}-{}-{}'.format(str(dt.year), str('%02d' % dt.month), str('%02d' % dt.day))
    dic_info_client[df_client['姓名'][i]]['报告日期'] = reportdate

In [97]:
dic_info_client

类别': '毒性，代谢'},
   '紫杉醇\tABCB1\trs1045642': {'基因型': 'AG', '证据等级': 3, '类别': '毒性'},
   '紫杉醇\tSOD2\trs4880': {'基因型': 'AA', '证据等级': 3, '类别': '毒性'},
   '紫杉醇\tERCC1\trs3212986': {'基因型': 'AC', '证据等级': 3, '类别': '毒性'},
   '卡铂\tERCC1\trs11615': {'基因型': 'AG', '证据等级': '2B', '类别': '疗效，毒性'},
   '卡铂\tXRCC1\trs25487': {'基因型': 'CC', '证据等级': '2B', '类别': '疗效'},
   '卡铂\tGSTP1\trs1695': {'基因型': 'AA', '证据等级': '2A', '类别': '毒性'},
   '奥沙利铂\tERCC1\trs11615': {'基因型': 'AG', '证据等级': '2B', '类别': '疗效，毒性'},
   '奥沙利铂\tXRCC1\trs25487': {'基因型': 'CC', '证据等级': '2B', '类别': '疗效'},
   '奥沙利铂\tGSTP1\trs1695': {'基因型': 'AA', '证据等级': '2A', '类别': '毒性'},
   '顺铂\tXPC\trs2228001': {'基因型': 'TT', '证据等级': '1B', '类别': '毒性'},
   '顺铂\tXRCC1\trs25487': {'基因型': 'CC', '证据等级': '2B', '类别': '疗效'},
   '顺铂\tERCC1\trs11615': {'基因型': 'AG', '证据等级': '2B', '类别': '疗效，毒性'},
   '顺铂\tGSTP1\trs1695': {'基因型': 'AA', '证据等级': '2A', '类别': '毒性'}},
  '报告日期': '2019-11-01'},
 '朱振芬': {'采样日期': '***--',
  '收样日期': '2019-10-15',
  '姓名': '朱振芬',
  '性别': '女',
  '年龄': 82.0,
 

In [98]:
document = Document('/Users/andy/Desktop/化疗药/各癌种手动模版/500基因化疗部分自动化模板1014.docx')#模版文件
table1 = document.tables[1]#需要替换的表格
list_drug = [i.text for i in table1.column_cells(0)]
list_index = [i for i,x in enumerate(list_drug) if x == '化疗药物']



def result_list(name, column):  #输入客户名字与指定列名，返回对应客户项目列表
    list_gt = []
    for i in dic_info_client[name]['基因型'].keys():
        list_gt.append(dic_info_client[name]['基因型'][i][column])
    
    list_drug = [i.text for i in table1.column_cells(0)]
    list_index = [i for i,x in enumerate(list_drug) if x == '化疗药物'] 
    for index in list_index:
        if index == 0:
            list_gt.insert(index, column)
        else:
            list_gt.insert(index, column)
            
    return list_gt




def clincal_list(name): 
    list_clincal = []
    for i in list_drug:
        if i == '化疗药物':
            list_clincal.append('临床提示')
        else:
            list_clincal.append(dic_info_client[name]['临床提示'][i])
    return list_clincal






In [99]:
list_drug

['化疗药物',
 '氟尿嘧啶+伊立替康+亚叶酸',
 '氟尿嘧啶+亚叶酸+奥沙利铂',
 '氟尿嘧啶+亚叶酸+奥沙利铂',
 '氟尿嘧啶+亚叶酸+奥沙利铂',
 '氟尿嘧啶+奥沙利铂',
 '氟尿嘧啶+亚叶酸',
 '氟尿嘧啶+亚叶酸',
 '氟尿嘧啶+亚叶酸',
 '替加氟+亚叶酸',
 '卡培他滨+奥沙利铂',
 '卡培他滨+伊立替康',
 '表柔比星+氟尿嘧啶+奥沙利铂',
 '表柔比星+氟尿嘧啶+奥沙利铂',
 '顺铂/卡铂+依托泊苷',
 '卡铂+多西他赛',
 '卡铂+多西他赛',
 '化疗药物',
 '顺铂+多西他赛',
 '顺铂+多西他赛',
 '顺铂+多西他赛',
 '顺铂/卡铂+吉西他滨',
 '顺铂/卡铂+吉西他滨',
 '顺铂/卡铂+培美曲塞',
 '顺铂/卡铂+培美曲塞',
 '顺铂/卡铂+长春瑞滨',
 '顺铂/卡铂+长春瑞滨',
 '顺铂/卡铂+长春瑞滨',
 '顺铂/卡铂+紫杉醇',
 '顺铂/卡铂+紫杉醇',
 '顺铂/卡铂+紫杉醇',
 '顺铂/卡铂+紫杉醇',
 '顺铂+环磷酰胺',
 '顺铂+环磷酰胺',
 '顺铂+环磷酰胺',
 '顺铂+环磷酰胺',
 '卡铂+环磷酰胺',
 '顺铂+多西他赛+吉西他滨+卡培他滨',
 '顺铂+表柔比星+吉西他滨+卡培他滨',
 '化疗药物',
 '顺铂/卡培他滨+紫杉醇',
 '顺铂+氟尿嘧啶',
 '顺铂+氟尿嘧啶',
 '环磷酰胺+表柔比星',
 '伊立替康',
 '伊立替康',
 '伊立替康',
 '环磷酰胺',
 '甲氨蝶呤',
 '蒽环类',
 '培美曲塞',
 '培美曲塞',
 '吉西他滨',
 '吉西他滨',
 '氟尿嘧啶',
 '氟尿嘧啶',
 '氟尿嘧啶',
 '替加氟',
 '替加氟',
 '卡培他滨',
 '卡培他滨',
 '多西他赛',
 '多西他赛',
 '多西他赛',
 '化疗药物',
 '紫杉醇',
 '紫杉醇',
 '紫杉醇',
 '卡铂',
 '卡铂',
 '卡铂',
 '奥沙利铂',
 '奥沙利铂',
 '奥沙利铂',
 '顺铂',
 '顺铂',
 '顺铂',
 '顺铂']

In [0]:
finall_dic = {}
for i in range(df_client.shape[0]):
    finall_dic.setdefault(df_client['姓名'][i], {})
    if str(df_client['采样时间'][i]) == '***':
        finall_dic[df_client['姓名'][i]]['采样日期'] = '***'
    else:
    
        finall_dic[df_client['姓名'][i]]['采样日期'] = str(df_client['采样时间'][i])[:4] + '-' + str(df_client['采样时间'][i])[4:6] + '-' + str(df_client['采样时间'][i])[6:8]
    finall_dic[df_client['姓名'][i]]['收样日期'] = str(df_client['收样时间'][i])[:4] + '-' + str(df_client['收样时间'][i])[4:6] + '-' + str(df_client['收样时间'][i])[6:8]
    finall_dic[df_client['姓名'][i]]['姓名'] = df_client['姓名'][i]
    finall_dic[df_client['姓名'][i]]['性别'] = df_client['性别'][i]
    finall_dic[df_client['姓名'][i]]['年龄'] = df_client['年龄'][i]
    finall_dic[df_client['姓名'][i]]['样本编号'] = df_client['实验室编号'][i]
    dt = datetime.date.today()
    reportdate = '{}-{}-{}'.format(str(dt.year), str('%02d' % dt.month), str('%02d' % dt.day))
    finall_dic[df_client['姓名'][i]]['报告日期'] = reportdate
    finall_dic[df_client['姓名'][i]]['临床提示'] = clincal_list(df_client['姓名'][i])
    finall_dic[df_client['姓名'][i]]['基因型'] = result_list(df_client['姓名'][i], '基因型')
    finall_dic[df_client['姓名'][i]]['类别'] = result_list(df_client['姓名'][i], '类别')

In [103]:
#file = '/Users/andy/Desktop/化疗药/各癌种手动模版/（模板）结直肠癌化疗用药报告模板.docx' #模版文件
file_name = '新500基因化疗报告.docx'
for name in finall_dic.keys():
    for i in range(0, len(table1.rows)):
        if table1.cell(i, 3).text == '基因型':
            pass
        else:
            table1.cell(i, 3).text = ''
            table1.cell(i, 5).text = ''
            table1.cell(i, 4).text = ''
    document.save(name + '-' + file_name)



    document = Document(name + '-' + file_name)
    table1 = document.tables[1]
    
    for i in range(0, len(table1.rows)):
        if table1.cell(i, 3).text == '基因型':
            pass
    
    
        else:
        
            run1 = table1.cell(i, 3).paragraphs[0].add_run(finall_dic[name]['基因型'][i])#修改基因型
            run2 = table1.cell(i, 4).paragraphs[0].add_run(finall_dic[name]['类别'][i])
            run1.font.size = Pt(10)
            run2.font.size = Pt(10)  
        #run = table1.cell(i, 3).paragraphs[0].add_run(res['检测结果'][i])
            if not table1.cell(i, 5).text:
                run = table1.cell(i, 5).paragraphs[0].add_run(finall_dic[name]['临床提示'][i]) #修改临床提示
            run.font.size = Pt(10)
            table1.cell(i,3).paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            table1.cell(i,5).paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            table1.cell(i,4).paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    document.save(name + '-' + file_name)



    
    
    
    tpl = DocxTemplate(name + '-' + file_name)
    tpl.render(finall_dic[name])
    tpl.save(name + '-' + file_name)
    
    
    
    
    
    
    
    
   
